# 🧪 Demo: Default Pipeline

This notebook demonstrates the default usage of the **Distance Matrix Generator**:
- Fetch travel data from the Google Maps API
- Generate a distance matrix
- Optionally store data in a SQL database
- Display the output as a pandas DataFrame

## 🛠️ Setup

To run the pipeline in Jupyter Notebook, we add the project root to `sys.path`.

We also import `pandas` for output preview and load the pipeline modules from the project.


In [1]:
import sys
import os

sys.path.append(os.path.abspath(".."))

import pandas as pd
from distance_matrix.fetcher import GoogleMapsFetcher
from distance_matrix.generator import DistanceMatrixGenerator

## 📍 Input Data

Create or load origin and destination data.

It's recommended to use dictionaries in the form `{label: location}` for better readability in the distance matrix and database output.


In [2]:
origin_dictionary = {
    "Port of Rotterdam": "Wilhelminakade 909, 3072 AP Rotterdam, Netherlands",
    "Port of Antwerp": "Loodglansstraat 5, 2030 Antwerpen, Belgium"
}

destination_dictionary = {
    "Port of Hamburg": "Am Ballinkai 1, 21129 Hamburg, Germany",
    "Port of Bremerhaven": "27580 Bremerhaven, Germany"
}

## 🌐 Fetch Travel Data

Use the `GoogleMapsFetcher` to request real-world travel data between all origin–destination pairs.

Results are stored as a JSON response with metadata.


In [3]:
fetcher = GoogleMapsFetcher(list(origin_dictionary.values()), list(destination_dictionary.values()))
fetcher.run_fetch_pipeline()
request_filename = fetcher.filename

Saved API response to ../data/raw/gmaps_dist_matrix_data_Wilhel_Loodgl_AmBall_922d24f.json


## 🏗️ Generate Distance Matrix

Use the `DistanceMatrixGenerator` to transform the raw API response into a structured CSV matrix and optionally store the data in a SQL database.


In [4]:
generator = DistanceMatrixGenerator(request_filename,
                                    list(origin_dictionary.keys()),
                                    list(destination_dictionary.keys()),
                                    write_to_db=True,
                                    dbms='mysql',
                                    db_name='test_distance_database')
generator.build_matrix()

DB CREATED READY TO CONTINUE
Data Frame has been written to ../data/processed/gmaps_dist_matrix_data_Wilhel_Loodgl_AmBall_922d24f.csv


## 📊 Preview Output

Load and display the generated distance matrix from the `data/processed/` directory using pandas.


In [5]:
directory = '../data/processed/'
export_path = os.path.join(directory, f"{request_filename}.csv")

df = pd.read_csv(export_path)
df.set_index("Matrix", inplace=True)
print(df)

                   Port Of Hamburg  Port Of Bremerhaven
Matrix                                                 
Port Of Rotterdam            491.0                416.0
Port Of Antwerp              548.0                475.0
